In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import time

In [ ]:
housing_data = pd.read_csv('Bangalore.csv')
housing_data.head() 

In [ ]:
varlist = ['Price','Area','NumBedrooms','Resale','MaintenanceStaff','Gymnasium','SwimmingPool','LandscapedGardens','IndoorGames']
data = housing_data[varlist].copy()

In [ ]:
# checking for null values
print(data.info())

# checking for outliers
print(data.describe())

In [ ]:
# some of the features like Gymnaisum should have bool values and the mean should not be more than 1
# plotting the to check if scaling is required
fig,ax = plt.subplots(1,8, figsize=(25,4), sharey = True)
for i in range(len(ax)):
    ax[i].scatter(data[varlist[i+1]],data[varlist[0]])
    ax[i].set_xlabel(varlist[i+1])
ax[0].set_ylabel('Price')

In [ ]:
# MainenanceStaff, Gymnasium, SwimmingPool, LandscapedGardens and IndoorGames needs to be manipulated
MaintenanceStaff = [1 if i>1 else 0 for i in data.MaintenanceStaff]
Gymnasium = [1 if i>1 else 0 for i in data.Gymnasium]
SwimmingPool = [1 if i>1 else 0 for i in data.SwimmingPool]
LandscapedGardens = [1 if i>1 else 0 for i in data.LandscapedGardens]
IndoorGames = [1 if i>1 else 0 for i in data.IndoorGames]

In [ ]:
# all the features are scaled except Area and NumBedrooms
# performing z-score normalization on Area
std_dev = np.std(data.Area)
mean = np.mean(data.Area)
Area = [(i-mean)/std_dev for i in data.Area]

# performing z-score normalization on NumBedrooms
std_dev = np.std(data.NumBedrooms)
mean = np.mean(data.NumBedrooms)
NumBedrooms = [(i-mean)/std_dev for i in data.NumBedrooms]

In [ ]:
# plotting Price to check if scaling is required
plt.scatter(data.Price,list(range(len(data.Price))))

In [ ]:
# some values are too large
# performing z-score normalization on price
std_dev = np.std(data.Price)
mean = np.mean(data.Price)
Price = [(i-std_dev)/mean for i in data.Price]
plt.scatter(Price,list(range(len(Price))))

In [ ]:
# combining scaled data into a single dataframe
scaled_data = pd.DataFrame()
for var in varlist:
    if var == 'Resale':
        scaled_data[var] = data.Resale
    else:
        scaled_data[var] = eval(var)

fig,ax = plt.subplots(1,8, figsize=(25,4), sharey = True)
for i in range(len(ax)):
    ax[i].scatter(scaled_data[varlist[i+1]],scaled_data[varlist[0]])
    ax[i].set_xlabel(varlist[i+1])
ax[0].set_ylabel('Price')

In [ ]:
# splitting the data into training and testing data
scaled_data = scaled_data.sample(frac=1, random_state=42).reset_index(drop=True)
train_to_test_ratio = 0.7
size = int(train_to_test_ratio * len(scaled_data))

data_train = scaled_data[size:]
data_test = scaled_data[:size]

x_train = data_train[varlist[1:]].values
y_train = data_train[varlist[0]].values
           

In [ ]:
#%% derived function
def predicted_y(x,w,b):
    '''
    
    Parameters
    ----------
    x : numpy.ndarray
        DESCRIPTION - array of features (x1...xn)
    w : numpy.ndarray
        DESCRIPTION - array of weights (w1...wn)
    b : float
        DESCRIPTION - bias
    Returns
    -------
    y : float
        DESCRIPTION - predicted y based on x

    '''
    f = np.dot(w,x) + b
    return f

In [ ]:
# cost function
def compute_cost(x_train,y_train,w,b):
    '''
    Parameters
    ----------
    x_train : numpy.ndarray
        DESCRIPTION - training data features
    y_train : numpy.ndarray
        DESCRIPTION : training data targets
    w : numpy.ndarray
        DESCRIPTION - array of weights (w1...wn)
    b : float
        DESCRIPTION - bias
    Returns
    -------
    y : float
        DESCRIPTION - predicted y based on x

    '''
    SUM = 0
    m = x_train.shape[0]
    for i in range(m):
        x = x_train[i]
        y = y_train[i]
        
        f = predicted_y(x,w,b)
        
        SUM += (f-y)**2
    cost = SUM/(2*m)
    return cost

In [ ]:
# compute gradient
def compute_gradient(x_train,y_train,w,b):
    '''
    Parameters
    ----------
    x_train : numpy.ndarray
        DESCRIPTION - training data features
    y_train : numpy.ndarray
        DESCRIPTION : training data targets
    w : numpy.ndarray
        DESCRIPTION - array of weights (w1...wn)
    b : float
        DESCRIPTION - bias
        
    Returns
    -------
    dj_dw : numpy.ndarray
        DESCRIPTION - array of gradient values for all parameters
    dj_db : numpy.float64
        DESCRIPTION - bias gradient
    '''
    SUMw = 0
    SUMb = 0
    
    m = x_train.shape[0]
    SUMw = np.zeros(8)
    SWMb = 0
    for i in range(m):
        x = x_train[i]
        y = y_train[i]

        f = predicted_y(x,w,b)
        
        SUMw += np.array([np.dot((f-y),i) for i in x])
        SUMb += f-y

    dj_dw = SUMw/m
    dj_db = SUMb/m
    
    return dj_dw, dj_db

In [ ]:
#gradient descent
def gradient_descent(x_train,y_train,num_iters,alpha,w,b):
    '''
    Parameters
    ----------
    x_train : numpy.ndarray
        DESCRIPTION - training data features
    y_train : numpy.ndarray
        DESCRIPTION : training data targets
    num_iters : int
        DESCRIPTION : number of iterations
    alpha : float
        DESCRIPTION - learning rate
    w : numpy.ndarray
        DESCRIPTION - array of weights (w1...wn)
    b : float
        DESCRIPTION - bias    
    
    
    '''
    J_history = []
    w_history = []
    b_history = []
    
    m = x_train.shape[0]
    
    
    for i in range(num_iters):
        dj_dw, dj_db = compute_gradient(x_train,y_train,w,b)
        w = w - alpha*dj_dw
        b = b - alpha*dj_db
        
        cost = compute_cost(x_train, y_train,w,b)
        J_history.append(cost)
        w_history.append(w)
        b_history.append(b)
        
        if i% math.ceil(num_iters/10) == 0:
            print(f"Iteration {i:4}: Cost {J_history[-1]:0.2e} ")
                 # f"dj_dw: {dj_dw}, dj_db: {dj_db: 0.3e}  ",
                 # f"w: {w}, b:{b: 0.5e}")
        # break the loop if the cost has converged i.e. the diference in cost is less than 1e-7
        if  i>1000 and  (J_history[-1] - J_history[-2] <= 1e-7):    
            print('\ncost has converged')
            break
    print('w = {}\nb = {}'.format(w,b))
    return J_history, w_history, b_history #return w and J,w history for graphing
        

In [ ]:
import time
tic = time.time()
w = np.random.normal(size=8)
b = 0
num_iters = 10000
alpha = 1e-2

# perform gradient descent
J_history, w_history, b_history = gradient_descent(x_train, y_train, num_iters, alpha, w, b)
toc = time.time()
time = (toc-tic)/60
print(f'time : {time}')

In [ ]:
f1 = plt.figure(1)
f1.add_subplot(3,1,1)
plt.plot(w_history[:][0],label = 'w')
plt.legend()
f1.add_subplot(3,1,2)
plt.plot(b_history,label='b')
plt.legend()
f1.add_subplot(3,1,3)
plt.plot(J_history,label = 'cost')
plt.legend()